In [120]:
import sqlite3
import datetime as dt
import pandas as pd
import re

In [340]:

class UnrecognisedRoleTypeError(Exception):
    pass

def update_db(
    q: str = None,
    db_name: str = 'ChoreTracker.sqlite3',
    commit: bool = True
    ):
        conn = sqlite3.connect(db_name) 
        cursor = conn.cursor()
        r = cursor.execute(q)
        if commit:
            conn.commit()
        conn.close()
        return r

def query_db(
    q: str = None,
    db_name: str = 'ChoreTracker.sqlite3',
    ):
        conn = sqlite3.connect(db_name) 
        cursor = conn.cursor()
        r = cursor.execute(q).fetchall()
        cols = [c[0] for c in cursor.description]
        conn.close()
        return pd.DataFrame(columns=cols, data=r)


In [92]:
reset = False

In [90]:
# set up the database
conn = sqlite3.connect('ChoreTracker.sqlite3')
cursor = conn.cursor()
tables = [t[0] for t in cursor.execute(""" SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name;  """).fetchall()]

In [91]:
tables

['choreinstances',
 'chorerates',
 'choreresponsibilities',
 'chores',
 'credentials',
 'people',
 'roles',
 'sqlite_sequence']

In [194]:
# Setting up the data model
reset=False
if reset:
    r = cursor.execute("""
        drop table if exists chores    
        """)
r = cursor.execute("""
create table if not exists chores (
    ChoreID integer primary key AUTOINCREMENT,
    name text, 
    schedule text, -- how frequently does this chore happen (in days)?
    start_date text, -- when is the first day that this chore applies?
    start_time text, -- what time of day does the chore window open?
    window text, -- how many hours before the chore needs to be finished? 
    repeats INTEGER, -- boolean to say whether this chore sould repeat or not
    active INTEGER -- boolean, is chore active?
    );
""")


In [195]:
class NoChoreNameSuppliedError(Exception):
    pass

class NoChoreIdSuppliedError(Exception):
    pass

class NoStartDateSuppliedError(Exception):
    pass

class NoStartTimeSuppliedError(Exception):
    pass

class IncorrectTimeWindowFormatError(Exception):
    pass

def create_chore(
    name: str = None,
    schedule: str = '1D',
    start_date: dt.date = dt.date.today(),
    start_time: dt.time = dt.time(7),
    window: str = '4H',
    repeats: bool = True,
    active: bool = True,
):
    if name is None:
        raise NoChoreNameSuppliedError
    if start_date is None:
        raise NoStartDateSuppliedError
    if start_time is None:
        raise NoStartTimeSuppliedError
    
    if not re.match(r'\d+[YWDMH]', schedule):
        raise IncorrectTimeWindowFormatError

    if not re.match(r'\d+[YWDMH]', window):
        raise IncorrectTimeWindowFormatError

    update_db(f"""
        INSERT INTO chores 
        (name, schedule, start_date, start_time, window, repeats, active)
        VALUES
        ('{name}', '{schedule}', '{start_date}', '{start_time}', '{window}', {repeats}, {active})
        """)
    
    return True

def delete_chore(choreid:int = None):
    if choreid is None:
        raise NoChoreIdSuppliedError

    update_db(f"""
        delete from chores where ChoreID = {choreid}
    """)
    return True

def get_chores():
    return query_db(q="""
        SELECT * FROM chores
    """)

In [204]:
if False: 
    create_chore(
    name='Two day today inactive test', 
    schedule='2D', 
    start_date=dt.date.today()-dt.timedelta(days=0), 
    repeats=True,
    active=False)

True

In [186]:
# delete_chore(choreid=6)

True

In [342]:
get_chores()

,ChoreID,name,schedule,start_date,start_time,window,repeats,active
0,1,Two week test,1W,2023-01-05,07:00:00,4H,1,1
1,2,One week test,1W,2023-01-12,07:00:00,4H,1,1
2,3,Today test,1W,2023-01-19,07:00:00,4H,1,1
3,4,Two day yesterday test,2D,2023-01-18,07:00:00,4H,1,1
4,5,Two day today test,2D,2023-01-19,07:00:00,4H,1,1
5,6,Two day today inactive test,2D,2023-01-19,07:00:00,4H,1,0


In [343]:
reset

False

In [344]:
if reset:
    r = cursor.execute("""
    drop table roles;
    """)
    
r = cursor.execute("""
create table if not exists roles (
    RoleID integer primary key AUTOINCREMENT,
    RoleName text UNIQUE -- parent or child
    );
""")

if reset:
    for role in ['Parent', 'Child']:
        r = cursor.execute(f"""
            INSERT INTO roles (RoleName) VALUES ('{role}');
        """)

conn.commit()

In [349]:
roles = query_db(""" select * from roles """) # .fetchall()
dict(zip(roles['RoleName'], roles['RoleID']))
# role_ids = dict([(dict(roles)[k], k) for k in dict(roles)])
# role_ids

{'Parent': 2, 'Child': 3}

In [51]:
if reset:
    r = cursor.execute(""" drop table people """)

In [49]:
# Keep track of individuals so chores can be assigned to them
r = cursor.execute("""
create table if not exists people (
    PersonID integer primary key AUTOINCREMENT,
    PersonName text, 
    RoleID INTEGER, -- whether person is parent or child
    CurrentBalance INTEGER -- balance in pence, display number/100 for proper currency
    );
""")


In [146]:

def create_person(PersonName: str = None, RoleType: str = None, CurrentBalance: int = 0):
    if not RoleType in role_ids:
        raise UnrecognisedRoleTypeError

    r = update_db(q=f"""
            INSERT INTO people (PersonName, RoleID, CurrentBalance) VALUES ('{PersonName}', {role_ids[RoleType]}, {CurrentBalance});
        """, commit=True)
    return True

In [209]:
def get_people():
    return query_db(q="""
        select * from people
    """)

In [211]:
create_person(
    PersonName='Lottie',
    RoleType='Child'
)

True

In [212]:
people_cols, people_data = get_people()
pd.DataFrame(columns=people_cols, data=people_data)

,PersonID,PersonName,RoleID,CurrentBalance
0,1,Mike,2,0
1,2,Michelle,2,0
2,3,Jake,3,0
3,4,Lottie,3,0


In [444]:
reset = False
if reset:
    r = cursor.execute("""
        drop table if exists choreinstances
    """)    
# We're going to have a daily scheduled process to calculate the day's chores, and pop them in this table
# so we can track whether repeated chores are carried out 
r = cursor.execute("""
create table if not exists choreinstances (
    ChoreInstanceID integer primary key,
    ChoreID INTEGER, 
    ChoreDate TEXT, -- the date the chore was due
    Completed INTEGER, -- Boolean
    CompletedBy INTEGER, -- this will store PersonID of whoever completed it
    Validated INTEGER, -- has it been checked by a parent?
    Rate INTEGER, -- chore rate when it was completed
    Banked INTEGER, -- has it been paid into the bank account?
    BankedDate TEXT -- when was it registered as paid?
    );
""")


In [445]:
def get_chore_instances():
    return query_db("""
        select * from choreinstances
    """)

In [446]:
get_chore_instances()

,ChoreInstanceID,ChoreID,ChoreDate,Completed,CompletedBy,Validated,Rate,Banked,BankedDate


In [447]:
def check_if_active(row: pd.Series) -> bool:
    retval = False
    test_date = pd.to_datetime(row['start_date']).date()

    if test_date == dt.date.today():
        retval = True

    elif row['repeats']:
        freq, unit = re.findall(r'(\d+)(\w)', row['schedule'])[0]
        if not isinstance(freq, int):
            freq=int(freq)
        # YMWDH
        if unit == 'H':
            interval = dt.timedelta(hours=freq)
        if unit == 'D':
            interval = dt.timedelta(days=freq)
        if unit == 'W':
            interval = dt.timedelta(days=freq * 7)
        if unit == 'M':
            # timedelta doens't do months, need
            # to handle this better
            interval = dt.timedelta(days=freq * 30)
        if unit == 'Y':
            interval = dt.timedelta(days=freq * 365)
        
        while test_date < dt.date.today():
            test_date += interval
            if test_date == dt.date.today():
                retval = True
                test_date = dt.date.today() + dt.timedelta(days=1)
                break


    return retval

In [448]:
def get_active_chores(
    chore_date: dt.date = None
):
    if chore_date is None:
        chore_date = dt.date.today()
    chore_df = query_db(
        """ SELECT * FROM chores where active = 1 """
    )
    chore_df['ActiveToday'] = chore_df.apply(check_if_active, axis=1)
    return chore_df[chore_df['ActiveToday'] == True]

In [449]:
# create_chore(name='One year test', schedule='1Y', start_date=dt.date.today()-dt.timedelta(days=365), repeats=True)

In [450]:
def update_choreinstances(
    chore_date: dt.datetime = None
):
    if chore_date is None:
        chore_date = dt.datetime.now()
    active_chores = get_active_chores(chore_date.date)
    update_db(f"""
        delete from choreinstances where date(ChoreDate)=date('{chore_date}')
    """)
    chore_rate = get_chore_rate(chore_date)
    # ['ChoreInstanceID',
    #  'ChoreID',
    #  'ChoreDate',
    #  'Completed',
    #  'CompletedBy',
    #  'Validated',
    #  'Rate',
    #  'Banked']
    completed = False
    validated = False
    banked = False
    for ChoreID in active_chores['ChoreID'].values:
        update_db(f"""
            INSERT INTO choreinstances 
            (ChoreID, ChoreDate, Completed, Validated, Rate, Banked)
            VALUES 
            ({ChoreID}, '{chore_date.date()}', {completed}, {validated}, {chore_rate}, {banked})
        """)
        


In [465]:
update_choreinstances()

In [466]:
get_chore_instances()

,ChoreInstanceID,ChoreID,ChoreDate,Completed,CompletedBy,Validated,Rate,Banked,BankedDate
0,1,1,2023-01-19,0,None,0,45,0,None
1,2,2,2023-01-19,0,None,0,45,0,None
2,3,3,2023-01-19,0,None,0,45,0,None
3,4,5,2023-01-19,0,None,0,45,0,None


In [467]:

# Who is responsible for each chore?
# More than one person can be responsible, so we have to be 
# able to track whether a multi-person chore has been completed and by whom
r = cursor.execute("""
create table if not exists choreresponsibilities (
    ResponsibilityID integer primary key,
    PersonID INTEGER, 
    ChoreID INTEGER
    );
""")


In [468]:
class UnrecognisedChoreIDError(Exception):
    pass

class UnrecognisedPersonIDError(Exception):
    pass


def set_responsibility(
    ChoreID: int = None,
    PersonID: int = None,
):
    if not ChoreID in get_chores()['ChoreID'].values:
        raise UnrecognisedChoreIDError
        
    if not PersonID in get_people()['PersonID'].values:
        raise UnrecognisedPersonIDError
        
    update_db(f""" 
        delete from choreresponsibilities 
        where 
        PersonID = {PersonID}
        and
        ChoreID = {ChoreID}
    """)

    update_db(f""" 
        INSERT INTO choreresponsibilities 
        (PersonID, ChoreID) 
        VALUES ({PersonID}, {ChoreID})
    """)

    return True

def get_responsibilities():
    return query_db(""" select * from choreresponsibilities """)
    

In [469]:
get_people() # , get_chores()

,PersonID,PersonName,RoleID,CurrentBalance
0,1,Mike,2,0
1,2,Michelle,2,0
2,3,Jake,3,0
3,4,Lottie,3,0


In [470]:
set_responsibility(PersonID=4, ChoreID=1)

True

In [471]:
get_responsibilities()

,ResponsibilityID,PersonID,ChoreID
0,1,2,6
1,3,2,2
2,4,3,3
3,5,4,4
4,6,4,5
5,7,4,6
6,8,1,1
7,9,4,1


In [472]:
query_db(f"""
    select  
        i.Choreid,
        i.Choreinstanceid,
        c.name,
        p.PersonName,
        p.personid,
        i.Rate,
        i.Completed,
        i.CompletedBy,
        i.Validated,
        i.Banked,
        i.BankedDate
    from choreinstances as i
    join chores as c
    on i.ChoreID = c.ChoreID
    join
    choreresponsibilities as r
    on i.ChoreID = r.ChoreID
    join
    people as p
    on p.PersonID = r.PersonID
    where i.choredate = date('{chore_date}')
    order by p.PersonName
""")


,ChoreID,ChoreInstanceID,name,PersonName,PersonID,Rate,Completed,CompletedBy,Validated,Banked,BankedDate
0,3,3,Today test,Jake,3,45,0,None,0,0,None
1,1,1,Two week test,Lottie,4,45,0,None,0,0,None
2,5,4,Two day today test,Lottie,4,45,0,None,0,0,None
3,2,2,One week test,Michelle,2,45,0,None,0,0,None
4,1,1,Two week test,Mike,1,45,0,None,0,0,None


In [473]:
class NoPersonIdSuppliedError(Exception):
    pass

class NoChoreInstanceIdSuppliedError(Exception):
    pass

class ChoreNotCompletedError(Exception):
    pass

class UnrecognisedChoreInstanceIDError(Exception):
    pass

def complete_chore_instance(
    chore_instance_id: int = None,
    person_id: int = None,
):
    if person_id is None:
        raise NoPersonIdSuppliedError

    if chore_instance_id is None:
        raise NoChoreInstanceIdSuppliedError


    chore_instance_df = query_db(f"""
        select * from choreinstances where choreinstanceid = {chore_instance_id} and Completed=0
    """)

    if not len(chore_instance_df) == 1:
        # This is not a valid, incomplete chore
        return False


    person_df = query_db(f""" 
        select r.* from choreresponsibilities as r
        join choreinstances as i
        on r.choreid = i.choreid
        where PersonId = {person_id} and i.choreinstanceid = {chore_instance_id}
    """)

    if not len(person_df) == 1:
        # This isn't a valid person/chore combination
        return False
    
    # We've established it's a valid, incomplete chore 
    # and the person is assigned to it, so they're allowed
    # to complete it
    update_db(f"""
        UPDATE choreinstances SET Completed = 1, CompletedBy={person_id} where 
        Choreinstanceid = {chore_instance_id}
    """)

    return True

    
def uncomplete_chore_instance(
    chore_instance_id: int = None,
):

    if chore_instance_id is None:
        raise NoChoreInstanceIdSuppliedError

    update_db(f"""
        UPDATE choreinstances SET Completed = 0, Validated=0, Banked=0, CompletedBy=NULL where 
        Choreinstanceid = {chore_instance_id}
    """)

    return True

   
def invalidate_chore_instance(
    chore_instance_id: int = None,
):

    if chore_instance_id is None:
        raise NoChoreInstanceIdSuppliedError

    update_db(f"""
        UPDATE choreinstances SET Validated=0 where 
        Choreinstanceid = {chore_instance_id}
    """)

    return True

def validate_chore_instance(
    chore_instance_id: int = None,
):

    if chore_instance_id is None:
        raise NoChoreInstanceIdSuppliedError

    chore_check = query_db(f"""
        select Completed from choreinstances where choreinstanceid={chore_instance_id}
        """)
    if not len(chore_check) > 0:
        raise UnrecognisedChoreInstanceIDError

    if not chore_check['Completed'].values.max() == 1:
        raise ChoreNotCompletedError

    # Probably need to check it's been done 
    # and validate chore instance id
    
    update_db(f"""
        UPDATE choreinstances SET Validated = 1
        where 
        Choreinstanceid = {chore_instance_id}
    """)

    return True

In [480]:
query_db(f"""
    select  
        i.Choreid,
        i.Choreinstanceid,
        c.name,
        p.PersonName,
        p.personid,
        i.Rate,
        i.Completed,
        i.CompletedBy,
        i.Validated,
        i.Banked,
        i.BankedDate
    from choreinstances as i
    join chores as c
    on i.ChoreID = c.ChoreID
    join
    choreresponsibilities as r
    on i.ChoreID = r.ChoreID
    join
    people as p
    on p.PersonID = r.PersonID
    where i.Validated =1 -- and i.Banked=0
    and i.CompletedBy = p.personId
    order by p.PersonName
""")

,ChoreID,ChoreInstanceID,name,PersonName,PersonID,Rate,Completed,CompletedBy,Validated,Banked,BankedDate
0,1,1,Two week test,Mike,1,45,1,1,1,1,2023-01-19


In [475]:
def bank_owing_amounts(
    person_id: int = None, 
    banked_date: dt.date = None
    ):
    # How do we roll this back if it's done by mistake?
    # maybe set a banked date
    if person_id is None:
        raise NoPersonIdSuppliedError

    if banked_date is None:
        banked_date = dt.date.today()

    update_db(f"""
        update choreinstances set banked=1, bankeddate='{banked_date}'
        where 
        completedby={person_id} and 
        validated = 1 and
        banked = 0
    """)

    return True
    

In [481]:
conn.commit()
conn.close()

In [479]:
bank_owing_amounts(person_id=1)

True

In [476]:
complete_chore_instance(chore_instance_id=1, person_id=1)

True

In [420]:
uncomplete_chore_instance(chore_instance_id=1)

True

In [477]:
validate_chore_instance(chore_instance_id=1)

True

In [425]:
invalidate_chore_instance(chore_instance_id=1)

True

In [102]:
# What's the going rate for a chore these days?
r = cursor.execute("""
create table if not exists chorerates (
    RateID integer primary key,
    ChoreRate INTEGER, -- amount in pence
    StartDate TEXT -- when did the rate take effect?
    );
""")

In [116]:
def update_chore_rate(ChoreRate: int = 25):
    update_db(f""" 
        INSERT INTO chorerates 
        (Rate, StartDate) 
        VALUES ({ChoreRate}, '{dt.datetime.strftime(dt.datetime.now(), "%Y-%m-%d %H:%M:%S")}')
        """)

def get_chore_rates():
    return query_db(""" SELECT * FROM chorerates""")

In [356]:
def get_chore_rate(chore_date: dt.datetime = None):
    if chore_date is None:
        chore_date = dt.datetime.now()
    return query_db(f""" 
        select Rate 
        from chorerates 
        join 
            (select max(datetime(StartDate)) as max_date from chorerates where datetime(StartDate) <= datetime('{chore_date}')) as m
        where datetime(StartDate) = m.max_date
        """)['Rate'].values.max()

In [357]:
get_chore_rate()

25

In [351]:
get_chore_rates()

,RateID,Rate,StartDate
0,1,25,2023-01-01
1,2,25,2023-01-01 19:39:00
2,3,30,2023-01-19 20:07:35
3,4,25,2023-01-19 20:07:59
4,5,35,2023-01-19 21:46:19
5,6,25,2023-01-19 21:47:35
6,7,35,2023-01-19 21:56:40
7,8,45,2023-01-19 21:57:13
8,9,55,2023-01-19 21:58:29
9,10,25,2023-01-19 21:58:38


In [374]:
update_chore_rate(ChoreRate=45)

In [66]:

# Need to have some way of storing the parent passcode
# and potentially change it.

r = cursor.execute("""
create table if not exists credentials (
    CredentialID integer primary key,
    RoleID INTEGER, -- need one or both of these two
    PersonID INTEGER, -- need one or both of these two
    Passcode TEXT -- this should possibly be encrypted...
    );
""")